In [1]:
import gym
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate
import matplotlib as plt


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#wrapper for cont env with PPO, plot result for several steps
def ppo_eval_interval(p, L, t_t, n_iter, n_step, gae, learning_rate=0.0003):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
    cont_env = make_vec_env('inventory_cont_disc_action_config_fix_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1, 
                     learning_rate = learning_rate,use_sde = False, gae_lambda = gae, n_steps = n_step)
    env_eval = make_vec_env('inventory_cont_disc_action_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=2*n_envs*n_step-1)#each iteration has 8192 timesteps with n_env=4
        timesteps = timesteps + 2*n_envs*n_step

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(-res_mean)
        res_std_arr.append(res_std)
        print(res_mean_arr)
    
#     plt.pyplot.plot(res_mean_arr)

    return res_mean_arr, res_std_arr

In [4]:
# listp = [0.25,1,4,9,39,99]
# listL = [30,50]
listp = [99]
listL = [1]
t_t = 2000000
n_iter = 10000 #for eval
# n_step = 8192
learning_rate = 0.0003
gae_lambda = 0.99
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        n_step = 64*L
        res_mean, res_std = ppo_eval_interval(p,L,t_t, n_iter, n_step, gae_lambda, learning_rate)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':min(res_mean), 'res_std': res_std[np.argmin(res_mean)]}, ignore_index=True)

Running PPO w/: p= 99 , L= 1
Using cpu device
-----------------------------
| time/              |      |
|    fps             | 2675 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 256  |
-----------------------------
mean:  -17.16072382939002
standard deviation: 2.8616204910410636
[17.16072382939002]
------------------------------------------
| time/                   |              |
|    fps                  | 7077         |
|    iterations           | 1            |
|    time_elapsed         | 0            |
|    total_timesteps      | 256          |
| train/                  |              |
|    approx_kl            | -0.001756601 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -3           |
|    explained_variance   | -0.0006      |
|    learning_rate        | 0.0003       |
|    loss                 | 1.75e+05     |
|    n_updates            | 10           |
|    policy_gr

mean:  -16.280696376233745
standard deviation: 1.548886580775257
[17.16072382939002, 16.260787893313584, 17.619872918613027, 15.92224055650309, 16.695635740402523, 17.488503725414503, 17.39953757849629, 18.95995685356672, 15.93481940749328, 16.280696376233745]
------------------------------------------
| time/                   |              |
|    fps                  | 4985         |
|    iterations           | 1            |
|    time_elapsed         | 0            |
|    total_timesteps      | 256          |
| train/                  |              |
|    approx_kl            | 0.0011074385 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.99        |
|    explained_variance   | -0.00114     |
|    learning_rate        | 0.0003       |
|    loss                 | 8.31e+04     |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.0018      |
|    value_loss           | 1.73e+05     |
--------

mean:  -32.77540819460283
standard deviation: 12.770490451551542
[17.16072382939002, 16.260787893313584, 17.619872918613027, 15.92224055650309, 16.695635740402523, 17.488503725414503, 17.39953757849629, 18.95995685356672, 15.93481940749328, 16.280696376233745, 19.067657903086996, 21.5680853542803, 24.308809380645815, 27.82911892692578, 24.732392818651533, 23.82322931845821, 30.908700310131405, 32.77540819460283]
-------------------------------------------
| time/                   |               |
|    fps                  | 3424          |
|    iterations           | 1             |
|    time_elapsed         | 0             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | -0.0030528875 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.99         |
|    explained_variance   | -0.0014       |
|    learning_rate        | 0.0003        |
|    loss   

mean:  -31.234618607118968
standard deviation: 4.2770170985131255
[17.16072382939002, 16.260787893313584, 17.619872918613027, 15.92224055650309, 16.695635740402523, 17.488503725414503, 17.39953757849629, 18.95995685356672, 15.93481940749328, 16.280696376233745, 19.067657903086996, 21.5680853542803, 24.308809380645815, 27.82911892692578, 24.732392818651533, 23.82322931845821, 30.908700310131405, 32.77540819460283, 40.11766543700171, 27.76597197718926, 37.28505597194101, 36.03287240564957, 26.283778153341053, 47.16177576856287, 31.234618607118968]
-------------------------------------------
| time/                   |               |
|    fps                  | 7166          |
|    iterations           | 1             |
|    time_elapsed         | 0             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | -0.0005811928 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    ent

mean:  -68.99478254834023
standard deviation: 12.244886991613807
[17.16072382939002, 16.260787893313584, 17.619872918613027, 15.92224055650309, 16.695635740402523, 17.488503725414503, 17.39953757849629, 18.95995685356672, 15.93481940749328, 16.280696376233745, 19.067657903086996, 21.5680853542803, 24.308809380645815, 27.82911892692578, 24.732392818651533, 23.82322931845821, 30.908700310131405, 32.77540819460283, 40.11766543700171, 27.76597197718926, 37.28505597194101, 36.03287240564957, 26.283778153341053, 47.16177576856287, 31.234618607118968, 28.39722787218428, 53.47792160946442, 57.51602371052072, 68.54004516930962, 66.42573167396753, 68.99478254834023]
-------------------------------------------
| time/                   |               |
|    fps                  | 7612          |
|    iterations           | 1             |
|    time_elapsed         | 0             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            |

mean:  -178.72308072330895
standard deviation: 47.345001704099666
[17.16072382939002, 16.260787893313584, 17.619872918613027, 15.92224055650309, 16.695635740402523, 17.488503725414503, 17.39953757849629, 18.95995685356672, 15.93481940749328, 16.280696376233745, 19.067657903086996, 21.5680853542803, 24.308809380645815, 27.82911892692578, 24.732392818651533, 23.82322931845821, 30.908700310131405, 32.77540819460283, 40.11766543700171, 27.76597197718926, 37.28505597194101, 36.03287240564957, 26.283778153341053, 47.16177576856287, 31.234618607118968, 28.39722787218428, 53.47792160946442, 57.51602371052072, 68.54004516930962, 66.42573167396753, 68.99478254834023, 73.9826174238581, 53.18791176922311, 130.44655532515662, 111.3286726798312, 164.31566627465668, 178.72308072330895]
--------------------------------------------
| time/                   |                |
|    fps                  | 5566           |
|    iterations           | 1              |
|    time_elapsed         | 0         

mean:  -209.88244078476868
standard deviation: 32.222938409433624
[17.16072382939002, 16.260787893313584, 17.619872918613027, 15.92224055650309, 16.695635740402523, 17.488503725414503, 17.39953757849629, 18.95995685356672, 15.93481940749328, 16.280696376233745, 19.067657903086996, 21.5680853542803, 24.308809380645815, 27.82911892692578, 24.732392818651533, 23.82322931845821, 30.908700310131405, 32.77540819460283, 40.11766543700171, 27.76597197718926, 37.28505597194101, 36.03287240564957, 26.283778153341053, 47.16177576856287, 31.234618607118968, 28.39722787218428, 53.47792160946442, 57.51602371052072, 68.54004516930962, 66.42573167396753, 68.99478254834023, 73.9826174238581, 53.18791176922311, 130.44655532515662, 111.3286726798312, 164.31566627465668, 178.72308072330895, 255.65598568651316, 241.78168253981366, 259.40222579182074, 327.2441138118517, 294.10808086382815, 209.88244078476868]
------------------------------------------
| time/                   |              |
|    fps     

mean:  -250.67199651751463
standard deviation: 90.10326035421144
[17.16072382939002, 16.260787893313584, 17.619872918613027, 15.92224055650309, 16.695635740402523, 17.488503725414503, 17.39953757849629, 18.95995685356672, 15.93481940749328, 16.280696376233745, 19.067657903086996, 21.5680853542803, 24.308809380645815, 27.82911892692578, 24.732392818651533, 23.82322931845821, 30.908700310131405, 32.77540819460283, 40.11766543700171, 27.76597197718926, 37.28505597194101, 36.03287240564957, 26.283778153341053, 47.16177576856287, 31.234618607118968, 28.39722787218428, 53.47792160946442, 57.51602371052072, 68.54004516930962, 66.42573167396753, 68.99478254834023, 73.9826174238581, 53.18791176922311, 130.44655532515662, 111.3286726798312, 164.31566627465668, 178.72308072330895, 255.65598568651316, 241.78168253981366, 259.40222579182074, 327.2441138118517, 294.10808086382815, 209.88244078476868, 165.84090784097893, 239.8686304050534, 229.166142048082, 208.39624187974354, 250.67199651751463]
---

KeyboardInterrupt: 

In [ ]:
ppo_res

In [ ]:
plt.pyplot.plot(res_mean)